#  Tunbert finetuning

In [8]:
!pip install datasets

In [1]:
import json
import pandas as pd
from datasets import Dataset

In [2]:
# loading train & test datasets
with open(r"datasets/data_train.json", "r", encoding="utf-8") as read_file:
    train = json.load(read_file)
with open(r"datasets/data_test.json", "r", encoding="utf-8") as read_file:
    test = json.load(read_file)

Genrate dictionary:
DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_ro599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num: 10570
    })
})

In [3]:
def generateDictionary(train,test):
    datasetdictionary={"train":[],"validation":[]}
    for d in train["data"]:
        num_rows=0
        title=d["title"]
        for p in d["paragraphs"]:
            context=p["context"]
            for q in p["qas"]:
                question=q["question"]
                id=q["id"]
                answers=[]
                for a in q["answers"]:
                    answers.append({"text":a["text"],"answer_start":a["answer_start"]})
                    num_rows+=1
        features={"id":id,"title":title,"context":context,"question":question,"answers":answers}
        datasetdictionary["train"].append({"features":features,"num_rows":num_rows})
    for d in test["data"]:
        num_rows=0
        title=d["title"]
        for p in d["paragraphs"]:
            context=p["context"]
            for q in p["qas"]:
                question=q["question"]
                id=q["id"]
                answers=[]
                for a in q["answers"]:
                    answers.append({"text":a["text"],"answer_start":a["answer_start"]})
                    num_rows+=1
        features={"id":id,"title":title,"context":context,"question":question,"answers":answers}
        datasetdictionary["validation"].append({"features":features,"num_rows":num_rows})
    return datasetdictionary

data=generateDictionary(train,test)

print(data["train"])

[{'features': {'id': '00002', 'title': 'الملسوقة', 'context': '\u200f  أنجح طريقة لتحضير الملسوقة    ملسوقة  سميد ارطب 250 غ دقيق أبيض 250 غ ملعقة صغيرة ملح ملعقتين زيت نباتي  كوبين ماء دافئ نخلط المكونات نصب القليل من الخليط و ندهن بالفرشاة على المقلاة  عندما تنضج نضعها في كيس بلاستيكي نحفظها في الثلاجة ', 'question': 'كيفاش نحضر الملسوقة؟\u200f', 'answers': [{'text': '\u200f نخلط المكونات نصب القليل من الخليط و ندهن بالفرشاة على المقلاة  عندما تنضج نضعها في كيس بلاستيكي نحفظها في الثلاجة ', 'answer_start': -1}]}, 'num_rows': 2}, {'features': {'id': '00004', 'title': 'العين السبنيورية التونسية', 'context': ' عين سبنيورية تونسية   لحم مفروم 500 غ بيض مسلوق 6 ملعقتان صغيرتان من مزيج التوابل ملعقة صغيرة ملح ملعقة صغيرة بابريكا نصف ملعقة صغيرة فلفل اسود ملعقة كبيرة بقدونس مجفف ملعقة كبيرة نعناع جاف نصف  ملعقة صغيرة هريسة ملعقتين كبيرتين زيت ثوم 4 فصوص في وعاء نخلط جميع المكونات باستثناء البيض ثم نعجن اللحم بالتوابل حتى نحصل على عجينة ناعمة نطبق اللحم المفروم على ورق الخبز لنحصل على مستطيل

In [4]:
def dict_to_dataframe(data):
    featureslist=[]
    for d in data:
        id=d["features"]["id"]
        title=d["features"]["title"]
        context=d["features"]["context"]
        question=d["features"]["question"]
        featurelist=[]
        featurelist.append(id)
        featurelist.append(title)
        featurelist.append(context)
        featurelist.append(question)
        for a in d["features"]["answers"]:
            ans={"text":a["text"],"answer_start":a["answer_start"]}
        if(a["answer_start"]!=-1):
            featurelist.append(ans)
            featureslist.append(featurelist)
    return (pd.DataFrame(featureslist,columns=['id', 'title', 'context', 'question', 'answers'] ))
dataset_train=dict_to_dataframe(data["train"])
dataset_test=dict_to_dataframe(data["validation"])
dataset_test.head()

,id,title,context,question,answers
0,00196,العجة بالمرقاز التونسية,العجة بالمرقاز من المطبخ التونسي العجة بالمرق...,كيفاش نطيب العجة بالمرقاز التونسية؟,{'text': ' في مقلاة نحط ملعقة الزيت نقطع البصل...
1,00200,المقفول تونسي,مقفول تونسي طريقة عمل مقفول تونسي أكلة تونسية ...,كيفاش نقدم المقفول تونسي؟,{'text': 'نطيبو بيض في الماء و نزينو بيه يقدم ...
2,00205,الملوخية التونسية,الملوخية التونسية من المطبخ التونسي طريقة طبخ ...,وين نجم نحط الملوخية التونسية؟,{'text': 'هذة الأكلة كل ما تخليها في الثلاجة ت...
3,00207,الكفتاجي التونسي,اكلة الكفتاجي التونسية اكلة الكفتاجي التونسية...,كيفاش نحضر الكفتاجي التونسي؟,{'text': 'نقشر كل من البطاطا الجريوات والبدنجا...
4,00210,السوفلي التونسي,طريقة عمل عجينة السوفلي بالصورالمقادير:3اكواب ...,كيفاش نطيب السوفلي التونسي؟,{'text': 'نضعها في صينية الفرن نغطي الوجه بصفا...


In [7]:
!pip install transformers

  Using cached transformers-4.46.0-py3-none-any.whl (10.0 MB)
  Using cached tokenizers-0.20.1-cp38-none-win_amd64.whl (2.4 MB)
  Using cached transformers-4.46.0-py3-none-any.whl (10.0 MB)


In [8]:
#Load pre_trained Arabic model tensorflow
from transformers import AutoTokenizer,AutoModelForQuestionAnswering

model_name="not-lain/TunBERT"
#model = BertModel.from_pretrained(model_name)

from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at not-lain/TunBERT and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'e

# Tokenization
The SQuAD data needs to be tokenized using BERT’s tokenizer before fine-tuning.()

In [9]:
def prepare_train_features(df, filename):
    questions = [q.strip() for q in df["question"]]
    context = [q.strip() for q in df["context"]]
    inputs = tokenizer(
            questions,
            context,
            max_length=384,
            truncation="only_second",
            return_offsets_mapping=True,
            padding="max_length",
        )

    offset_mapping = inputs.pop("offset_mapping")

    start_positions = []
    end_positions = []
    answers = df['answers']
    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"]
        end_char = answer["answer_start"] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    df["start_positions"] = start_positions
    df["end_positions"] = end_positions


    data = {'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask'],
            'start_positions':start_positions,
            'end_positions': end_positions,
           }
    df = pd.DataFrame(data)
    df.to_csv(filename,index=False)
    return df

In [10]:
train_set = Dataset.from_pandas(prepare_train_features(dataset_train, 'encoding_train.csv'))
test_set=Dataset.from_pandas(prepare_train_features(dataset_test,'encoding_test.csv'))

# Finetuning Tunbert

In [11]:
!pip install accelerate==0.26.0


In [12]:
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator()

from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="jo3t",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()
trainer.save_model("jo3t_model_final")

<ipython-input-12-46020cf15d7c>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,4.292699
2,No log,3.997191
3,No log,3.964447


In [60]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer

# Load the fine-tuned model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("jo3t_model")
model = AutoModelForQuestionAnswering.from_pretrained("jo3t_model")

# Create the Trainer
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
)

# Evaluate the model
results = trainer.evaluate(test)

# Print the evaluation results
print(results)


<ipython-input-60-598cb1f1e41f>:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided []

In [56]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Load the fine-tuned model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("jo3t_model")
model = AutoModelForQuestionAnswering.from_pretrained("jo3t_model")

# Define a sample Arabic context and question
context = "في تونس، الأكلة الحارة تشمل الهريسة، وهي عجينة فلفل أحمر حار تستعمل في عدة أطباق."
question = "ما هي الأكلة الحارة في تونس؟"


# Tokenize input
inputs = tokenizer(question, context, return_tensors="pt")

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

# Extract answer tokens
answer_start_index = torch.argmax(outputs.start_logits)
answer_end_index = torch.argmax(outputs.end_logits) + 1
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start_index:answer_end_index]))

# Print the question and answer
print(f"Question: {question}")
print(f"Answer: {answer}")


Question: ما هي الأكلة الحارة في تونس؟
Answer: [CLS]


# Hyper-param tuning config